In [2]:
# 5. Import the required modules

import networkx as nx
import nx_arangodb as nxadb

from arango import ArangoClient

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

import os
import networkx as nx
import nx_arangodb as nxadb

os.environ["DATABASE_HOST"] = "https://b61c3b83bfe6.arangodb.cloud:8529"
os.environ["DATABASE_USERNAME"] = "root"
os.environ["DATABASE_PASSWORD"] = "RHr0KzkRUVlp61IisH8G"
os.environ["DATABASE_NAME"] = "DAC_devops_log"

db = ArangoClient(hosts="https://b61c3b83bfe6.arangodb.cloud:8529") \
    .db(username="root", 
        password="RHr0KzkRUVlp61IisH8G", 
        name="DAC_devops_log",
        verify=True)

[20:24:36 -0500] [INFO]: NetworkX-cuGraph is unavailable: No module named 'cupy'.


In [3]:
arango_graph = ArangoGraph(db)
G_adb = nxadb.DiGraph(name="tasks_sprint1")

[20:24:40 -0500] [INFO]: Graph 'tasks_sprint1' exists.
[20:24:40 -0500] [INFO]: Default node type set to 'task'


In [5]:
for node in G_adb.nodes():
    story_points = G_adb.nodes[node].get("StoryPoints", "1")  # Default to 1 if missing
    G_adb.nodes[node]["duration"] = int(story_points)  # Ensure integer conversion

In [6]:
# Find the longest path (Critical Path)
critical_path = nx.algorithms.dag.dag_longest_path(G_adb, weight="duration")

# Compute total project duration
critical_path_length = sum(G_adb.nodes[node]["duration"] for node in critical_path)

print("Critical Path:", critical_path)
print("Total Project Duration:", critical_path_length)

[20:25:46 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.33601975440979s


Critical Path: ['task/TSK-D5031D4A', 'task/TSK-B600DD62', 'task/TSK-29CE08AD', 'task/TSK-89BBB564', 'task/TSK-C2D38D23', 'task/TSK-FBD87AD9', 'task/TSK-7E9F09A9', 'task/TSK-A0500206', 'task/TSK-5A434477']
Total Project Duration: 45


In [11]:
# Step 1: Compute earliest start (ES) and finish (EF)
es = {}  # Earliest Start
ef = {}  # Earliest Finish

for node in nx.topological_sort(G_adb):
    es[node] = max((ef.get(pred, 0) for pred in G_adb.predecessors(node)), default=0)
    ef[node] = es[node] + G_adb.nodes[node]["duration"]

# Step 2: Compute latest finish (LF) and start (LS)
lf = {}  # Latest Finish
ls = {}  # Latest Start

# Start from the last node in the topological order
project_duration = max(ef.values())  # Total project duration
for node in reversed(list(nx.topological_sort(G_adb))):
    lf[node] = min((ls.get(succ, project_duration) for succ in G_adb.successors(node)), default=project_duration)
    ls[node] = lf[node] - G_adb.nodes[node]["duration"]

# Step 3: Compute slack time
slack = {node: ls[node] - es[node] for node in G_adb.nodes()}

# Print Slack times
for node, s in slack.items():
    print(f"Task {node}: Slack Time = {s}")

[20:30:36 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.530318260192871s
INFO:nx_arangodb:Graph 'tasks_sprint1' load took 2.530318260192871s
[20:30:38 -0500] [INFO]: Graph 'tasks_sprint1' load took 2.7433159351348877s
INFO:nx_arangodb:Graph 'tasks_sprint1' load took 2.7433159351348877s


Task task/TSK-FAC99DF1: Slack Time = 19
Task task/TSK-E2842052: Slack Time = 26
Task task/TSK-FAAA4D51: Slack Time = 16
Task task/TSK-A3E27D20: Slack Time = 50
Task task/TSK-C440EB07: Slack Time = 23
Task task/TSK-3BA37747: Slack Time = 19
Task task/TSK-E73A7D25: Slack Time = 11
Task task/TSK-F7E19B69: Slack Time = 11
Task task/TSK-10847035: Slack Time = 26
Task task/TSK-91119F5B: Slack Time = 47
Task task/TSK-991E6E11: Slack Time = 47
Task task/TSK-2F459E7F: Slack Time = 36
Task task/TSK-89754CBF: Slack Time = 21
Task task/TSK-A6994DD6: Slack Time = 34
Task task/TSK-D5031D4A: Slack Time = 7
Task task/TSK-2B1FAB57: Slack Time = 0
Task task/TSK-CB292DC1: Slack Time = 12
Task task/TSK-DC3ED3E2: Slack Time = 32
Task task/TSK-E2E87C0B: Slack Time = 19
Task task/TSK-98C9448B: Slack Time = 2
Task task/TSK-A0CCA417: Slack Time = 14
Task task/TSK-0885F3E9: Slack Time = 44
Task task/TSK-BE4188D4: Slack Time = 0
Task task/TSK-BE9555AC: Slack Time = 14
Task task/TSK-42E44473: Slack Time = 14
Task

In [12]:
print(lf)

{'task/TSK-5A434477': 52, 'task/TSK-A0500206': 47, 'task/TSK-42E44473': 52, 'task/TSK-FAAA4D51': 52, 'task/TSK-A8A24727': 52, 'task/TSK-85D5D00C': 52, 'task/TSK-98C9448B': 52, 'task/TSK-CB0C0404': 52, 'task/TSK-F567DBE1': 52, 'task/TSK-7E9F09A9': 39, 'task/TSK-E6CFDD02': 52, 'task/TSK-5C2187CF': 47, 'task/TSK-C891B895': 52, 'task/TSK-B03EF789': 39, 'task/TSK-FBD87AD9': 34, 'task/TSK-DFB54467': 34, 'task/TSK-427652F3': 47, 'task/TSK-0115FED3': 52, 'task/TSK-AE8CFDEC': 52, 'task/TSK-CB292DC1': 52, 'task/TSK-A0CCA417': 52, 'task/TSK-983C30DC': 42, 'task/TSK-C2D38D23': 26, 'task/TSK-3BA37747': 42, 'task/TSK-736E3B28': 52, 'task/TSK-1B43CB90': 52, 'task/TSK-F7E19B69': 42, 'task/TSK-B3B5A8F6': 52, 'task/TSK-A6994DD6': 52, 'task/TSK-763CE1C3': 34, 'task/TSK-89BBB564': 24, 'task/TSK-10847035': 52, 'task/TSK-FAC99DF1': 34, 'task/TSK-EBA1C369': 52, 'task/TSK-C4D5CB75': 52, 'task/TSK-1FE396B7': 35, 'task/TSK-FB3E490E': 41, 'task/TSK-54F9521F': 26, 'task/TSK-29CE08AD': 16, 'task/TSK-59CE546C': 26,